In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

# File location and type
file_location = "/FileStore/tables/colombo_cafes.csv"
file_type = "csv"

# Schema
schema = StructType([
  StructField("Title", StringType()),
  StructField("Total_score", FloatType()),
  StructField("Reviews_count", IntegerType()),
  StructField("Street", StringType()),
  StructField("City", StringType()),
  StructField("Country_code", StringType()),
  StructField("Website", StringType()),
  StructField("Phone", StringType()),
  StructField("Category_name", StringType())
])

# CSV options
infer_schema = "false"
first_row_is_header = "True"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
CoffeeData = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .schema(schema) \
  .load(file_location)

CoffeData.show(10)

+--------------------+-----------+-------------+--------------------+-------+------------+--------------------+---------------+-------------+---------+
|               Title|Total_score|Reviews_count|              Street|   City|Country_code|             Website|          Phone|Category_name|  Country|
+--------------------+-----------+-------------+--------------------+-------+------------+--------------------+---------------+-------------+---------+
|Coco Cafe- Street...|        4.6|           11|VVQ5+326, Elfinda...|Colombo|          LK|https://www.faceb...|+94 76 777 2225|         Cafe|Sri Lanka|
|         Cafe Celo09|   4.258885|            0|            VVRC+8M5|Colombo|          LK|                null|           null|  Coffee shop|Sri Lanka|
|Cafe 1959 by Raux...|        4.4|          248|        746 Galle Rd|Colombo|          LK|https://cafe-1959...|+94 112 590 520|  Coffee shop|Sri Lanka|
|            Cafe Lab|        2.8|            8|                null|Colombo|          L

In [0]:
CountryCodes = spark.read.format('CSV') \
    .option('inferSchema', 'True') \
    .option('header', 'True') \
    .option('sep', ',') \
    .load('/FileStore/tables/countryCodes.csv')

CountryCodes.show(10)

+-------------------+----+
|               Name|Code|
+-------------------+----+
|        Afghanistan|  AF|
|            Albania|  AL|
|            Algeria|  DZ|
|     American Samoa|  AS|
|            Andorra|  AD|
|             Angola|  AO|
|           Anguilla|  AI|
|         Antarctica|  AQ|
|Antigua and Barbuda|  AG|
|          Argentina|  AR|
+-------------------+----+
only showing top 10 rows



### Análise Exploratória

In [0]:
# Busca por possiveis problemas nos dados.
for column in CoffeeData.columns:
    CoffeeData.describe(column).show()

+-------+-------------------+
|summary|              Title|
+-------+-------------------+
|  count|                343|
|   mean|               null|
| stddev|               null|
|    min|            41 cafe|
|    max|ටිරොන්ගේ වඩේ තට්ටුව|
+-------+-------------------+

+-------+------------------+
|summary|       Total_score|
+-------+------------------+
|  count|               343|
|   mean| 4.258885004082504|
| stddev|0.6005719633166074|
|    min|               1.0|
|    max|               5.0|
+-------+------------------+

+-------+------------------+
|summary|     Reviews_count|
+-------+------------------+
|  count|               343|
|   mean|191.28862973760934|
| stddev|482.36961522161863|
|    min|                 0|
|    max|              3505|
+-------+------------------+

+-------+--------------------+
|summary|              Street|
+-------+--------------------+
|  count|                 340|
|   mean|                null|
| stddev|                null|
|    min|#L1-19, Th

In [0]:
# Visualização de valores nulos
for coluna in CoffeeData.columns:
    print(coluna, CoffeeData.filter(CoffeeData[coluna].isNull()).count())

Title 0
Total_score 0
Reviews_count 0
Street 3
City 0
Country_code 0
Website 246
Phone 101
Category_name 0


In [0]:
from pyspark.sql.functions import col, desc

totalLinhas01 = CoffeeData.filter(col('Total_score') >= 4.5 ).orderBy(desc('Total_score')).count()
totalLinhas02 = CoffeeData.filter(col('Total_score') >= 4 ).orderBy(desc('Total_score')).count()
totalLinhas03 = CoffeeData.filter(col('Total_score') >= 3 ).orderBy(desc('Total_score')).count()
totalLinhas04 = CoffeeData.filter(col('Total_score') >= 2 ).orderBy(desc('Total_score')).count()

print(totalLinhas01, totalLinhas02, totalLinhas03, totalLinhas04)

116 281 334 340


In [0]:
#Union das tabelas por meio do codigo do pais
CountryCodes = CountryCodes.withColumnRenamed('Name', 'Country').withColumnRenamed('Code', 'Country_code')

CoffeData = CoffeeData.join(CountryCodes, CoffeeData.Country_code == CountryCodes.Country_code, 'Inner') \
    .select(CoffeeData["*"], CountryCodes["Country"])

CoffeData.show(10)

+--------------------+-----------+-------------+--------------------+-------+------------+--------------------+---------------+-------------+---------+
|               Title|Total_score|Reviews_count|              Street|   City|Country_code|             Website|          Phone|Category_name|  Country|
+--------------------+-----------+-------------+--------------------+-------+------------+--------------------+---------------+-------------+---------+
|Coco Cafe- Street...|        4.6|           11|VVQ5+326, Elfinda...|Colombo|          LK|https://www.faceb...|+94 76 777 2225|         Cafe|Sri Lanka|
|         Cafe Celo09|   4.258885|            0|            VVRC+8M5|Colombo|          LK|                null|           null|  Coffee shop|Sri Lanka|
|Cafe 1959 by Raux...|        4.4|          248|        746 Galle Rd|Colombo|          LK|https://cafe-1959...|+94 112 590 520|  Coffee shop|Sri Lanka|
|            Cafe Lab|        2.8|            8|                null|Colombo|          L

In [0]:
from pyspark.sql.functions import count

CoffeeData.groupBy('City').agg(count('*')).show()

+--------------------+--------+
|                City|count(1)|
+--------------------+--------+
|            Colpetty|       1|
|             Kegalle|       1|
|            Kaduwela|       1|
|             Colombo|     334|
|                Mall|       1|
|         nedolakanda|       1|
|Sri Jayawardenepu...|       1|
|            Nugegoda|       1|
|        Urugodawatta|       1|
|   Port City Colombo|       1|
+--------------------+--------+



In [0]:
from pyspark.sql.functions import mean
CoffeeData.groupBy('City').agg(mean('Total_score')).show()

+--------------------+------------------+
|                City|  avg(Total_score)|
+--------------------+------------------+
|            Colpetty| 4.199999809265137|
|             Kegalle| 4.258884906768799|
|            Kaduwela|               5.0|
|             Colombo|4.2664065468097165|
|                Mall| 4.400000095367432|
|         nedolakanda| 4.258884906768799|
|Sri Jayawardenepu...|               4.5|
|            Nugegoda| 3.200000047683716|
|        Urugodawatta|               5.0|
|   Port City Colombo|               1.0|
+--------------------+------------------+



In [0]:
CoffeData.groupBy('Category_name').agg(count('*')).orderBy(desc(count('*'))).show()

+--------------------+--------+
|       Category_name|count(1)|
+--------------------+--------+
|                Cafe|     162|
|         Coffee shop|      73|
|          Restaurant|      57|
|              Bakery|       7|
|Vegetarian restau...|       5|
|Fast food restaurant|       3|
|           Tea house|       3|
|      Ice cream shop|       2|
|         Pastry shop|       2|
|    Asian restaurant|       2|
|  Takeout Restaurant|       2|
|Sri Lankan restau...|       2|
|North Eastern Ind...|       1|
|            Dog cafe|       1|
|           Cafeteria|       1|
|   Brunch restaurant|       1|
|   Korean restaurant|       1|
|    Pizza restaurant|       1|
|                 Bar|       1|
|          Game store|       1|
+--------------------+--------+
only showing top 20 rows



In [0]:
CoffeData.select('Title','Reviews_count','Total_score').orderBy(desc('Reviews_count')).show()

+--------------------+-------------+-----------+
|               Title|Reviews_count|Total_score|
+--------------------+-------------+-----------+
|          Tea Avenue|         3505|        4.3|
|The Coffee Bean &...|         3387|        4.3|
|     Café on the 5th|         3083|        4.2|
|      The Barnesbury|         2550|        4.3|
|    The Gallery Café|         2258|        4.4|
|     Caramel Pumpkin|         2253|        4.3|
|         The Embazzy|         2240|        4.3|
|Cafe Mocha by Bar...|         1649|        4.3|
|     Peppermint Cafe|         1641|        4.1|
|Java Lounge - Bam...|         1621|        4.4|
|Java Lounge Barne...|         1471|        4.4|
|     Cafe Noir Blanc|         1291|        4.3|
|         Cafe Kumbuk|         1210|        4.5|
|London House Of C...|         1190|        4.4|
|        THE CAULDRON|         1186|        4.5|
|Playtrix Sports B...|         1182|        4.3|
|Chocoholics - Hav...|         1163|        4.4|
|    The Central Per